In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/img.jpg
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/labels_map.txt

In [ ]:

root = '/gdrive/My Drive/CS492I/project'

In [ ]:
import os
import numpy as np
import time
from pathlib import Path

import json
from PIL import Image

import torch
from torchvision import transforms

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets.vision import StandardTransform
from torchvision.utils import make_grid
from torchvision import transforms
from torch.optim import SGD




!pip install efficientnet_pytorch
# from efficientnet_pytorch import EfficientNet
# model = EfficientNet.from_pretrained('efficientnet-b3')

torch.backends.cudnn.benchmark = True

!pip install git+https://github.com/lucasb-eyer/pydensecrf.git
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=6e87f11e5ae3acc4a664e50c0de79162a9f3f5460b570b56fec473f8ac322b13
  Stored in directory: /root/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet-pytorch


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-mza5qftp
  Running command git clone -q https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-mza5qftp
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pydensecrf: filename=pydensecrf-1.0rc2-cp38-cp38-linux_x86_64.whl size=2845209 sha256=532672ec82aca6b7db2c28b155742e6baf593423b7480dddc267f0a3fd7990fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-u0da1jc6/wheels/45/56/7a/826b4f5cd8459926ff5996ba9994fc36672487b6d2fa50d3d4
Successfully built pydensecrf


In [70]:
# model_name = 'efficientnet-b4'
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b4')
image_size = 224
print(image_size)
# image_size = EfficientNet.get_image_size(model_name) # 224

Loaded pretrained weights for efficientnet-b4
224


In [71]:
img_list = []
img1 = Image.open('/gdrive/MyDrive/project/car-damage-dataset/data1a/validation/00-damage/0001.JPEG')
img2 = Image.open('/gdrive/MyDrive/project/car-damage-dataset/data1a/validation/00-damage/0002.JPEG')
img3 = Image.open('/gdrive/MyDrive/project/car-damage-dataset/data1a/validation/00-damage/0003.JPEG')
img4 = Image.open('/gdrive/MyDrive/project/car-damage-dataset/data1a/validation/01-whole/0001.jpg')
img5 = Image.open('/gdrive/MyDrive/project/car-damage-dataset/data1a/validation/01-whole/0002.jpg')
img6 = Image.open('/gdrive/MyDrive/project/car-damage-dataset/data1a/validation/01-whole/0003.jpg')
img_list.extend([img1,img2,img3,img4,img5,img6])


In [72]:
# our image dataset
root1= '/gdrive/MyDrive/project/car-damage-dataset/'
folder_name_a_1 = root1+'data1a/training/00-damage/'
folder_name_a_2 =  root1+'data1a/training/01-whole/'
folder_name_a_3 =  root1+'data1a/validation/00-damage/'
folder_name_a_4 = root1+'data1a/validation/01-whole/'

folder_name_b_1 =  root1+'data2a/training/00-front/'
folder_name_b_2 =  root1+'data2a/training/01-rear/'
folder_name_b_3 =  root1+'data2a/training/02-side/'
folder_name_b_4 =  root1+'data2a/validation/00-front/'
folder_name_b_5 =  root1+'data2a/validation/01-rear/'
folder_name_b_6 =  root1+'data2a/validation/02-side/'

folder_name_c_1 =  root1+'data3a/training/01-minor/'
folder_name_c_2 =  root1+'data3a/training/02-moderate/'
folder_name_c_3 =  root1+'data3a/training/03-severe/'
folder_name_c_4 =  root1+'data3a/validation/01-minor/'
folder_name_c_5 =  root1+'data3a/validation/02-moderate/'
folder_name_c_6 =  root1+'data3a/validation/03-severe/'
folder_list = [folder_name_a_1,folder_name_a_2,folder_name_a_3,folder_name_a_4,folder_name_b_1,folder_name_b_2,folder_name_b_3,folder_name_b_4,folder_name_b_5,folder_name_b_6,folder_name_c_1,folder_name_c_2,folder_name_c_3,folder_name_c_4,folder_name_c_5,folder_name_c_6 ]


In [73]:
# storing all images in a list
# On google drive there are RAM restrictions that stop us from creating too big dictionaries.
# Therefore for drive I only use the data of my first 4 folders. 
# Furthermore, ~500 images are enough to cover all classes that represent cars in the imagenet dataset. (There arent more than 50 classes that describe cars) 
# So, 1150 images with their top2 classes for each is more than enough for our task. 
img_list=[]
for folder in folder_list[0:4:2]:
  os_list = os.listdir(folder)
  for _,img_path in enumerate(os_list):
    img_list.append(Image.open(folder+img_path))


In [74]:
# Transformation applie to all images and then store the images in img_list_processed

transform  = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.5,), (0.5,))
])
img_list_processed = []
for img in img_list:
  
  img_processed = transform(img).unsqueeze(0)

  img_list_processed.append(img_processed )

In [75]:
def preprocess(image):
  transform  = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
  processed_image = transform(image).unsqueeze(0)
  return processed_image

In [76]:
print(len(img_list_processed))
list_len = len(img_list_processed)

1150


In [77]:
# 1000 imagenet labels
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

In [ ]:
# Classify with EfficientNet
# Check the top 3 imagenet classes of prediction for 50 images 
model.eval()
for number in range(50):
  with torch.no_grad():
      logits = model(img_list_processed[number])
  preds = torch.topk(logits, k=3).indices.squeeze(0).tolist()
  result = [ (labels_map[index], ('{:.3f}').format(torch.softmax(logits,dim=1)[0,index].item())  )  for index in preds]
  print('\n',result)


In [ ]:
#  let's find most used imagenet classifiers that describe cars. then we will filter out top 15 of them
#  We classify all of our processed images and store into a hashmap the class as key and the class_probability as value

In [79]:
def total_car_categories():
  hashmap = {}
  for processed_image in img_list_processed:
    with torch.no_grad():
      output = model(processed_image)
    top_preds = torch.topk(output, k=2).indices.squeeze(0).tolist()
    result = [ (labels_map[index], torch.softmax(output,dim=1)[0,index].item() ) for index in top_preds]
    # print(result)

    for tupl in result:
      try:
        hashmap.update({tupl[0]:hashmap[tupl[0]]+tupl[1]})
      except KeyError:
        hashmap[tupl[0]] =  tupl[1]
  return hashmap
    # print('\n',hashmap,'\n')



In [80]:
map_categories = total_car_categories()

In [ ]:
# Now I only save top 15 classes that represent cars
max(map_categories, key = map_categories.get)
top_classes = sorted(map_categories, key = map_categories.get, reverse=True)[:30]


In [82]:
print(top_classes)

['pickup, pickup truck', 'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon', 'car wheel', 'jeep, landrover', 'limousine, limo', 'minivan', 'tow truck, tow car, wrecker', 'convertible', 'racer, race car, racing car', 'snowplow, snowplough', 'grille, radiator grille', 'half track', 'cab, hack, taxi, taxicab', 'sports car, sport car', 'ambulance', 'police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria', 'tank, army tank, armored combat vehicle, armoured combat vehicle', 'recreational vehicle, RV, R.V.', 'amphibian, amphibious vehicle', 'Model T', 'minibus', 'harvester, reaper', 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi', 'moving van', 'tractor', 'snowmobile', 'car mirror', 'motor scooter, scooter', 'golfcart, golf cart', 'photocopier']


In [81]:
# Now store the list of top classes
import json
path = '/gdrive/MyDrive/project/'

# Serialize data into file:
json.dump( top_classes, open( path+"car_detection.json", 'w' ) )


In [ ]:
# data = json.load( open( path+"file_name.json" ) )
# #


In [ ]:
import urllib

from urllib.request import Request, urlretrieve  # Python 3
def car_categories(image_path, cat_list):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    img = preprocess(Image.open('save.jpg'))

    with torch.no_grad():
      output = model(img)
    top_preds = torch.topk(output, k=5).indices.squeeze(0).tolist()
    result = [ (labels_map[index], torch.softmax(output,dim=1)[0,index].item() ) for index in top_preds]

    
    print ("Validating that this is a picture of your car...")
    for category in result:
        # print(top)
        if category[0] in cat_list:
            print (category[0], 'with percentage:', category[1])
            return "The model predicts this is a car"
    return "Not a car picture. Please try again."


In [ ]:
car_categories( '',top_classes)

Validating that this is a picture of your car...


'Not a car picture. Please try again.'